<a href="https://colab.research.google.com/github/YianKim/2022_PythonStudy_face_emotion_recognition/blob/main/uncertainty_%EA%B8%B0%EB%B0%98_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글 드라이브


In [178]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 라이브러리 불러오기


In [179]:
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import PIL
import pickle
import random
from tqdm import tqdm

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import LSTM
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import *
from keras.models import load_model

from tqdm import tqdm

## 데이터 불러오기


In [180]:
fashion_mnist = keras.datasets.fashion_mnist
(data, labels), (test_data, test_labels) = fashion_mnist.load_data()

In [181]:
train_data = data[range(20000)].reshape([20000,28,28,1])
train_labels = labels[range(20000)].reshape([20000,1])

In [182]:
unlab_data = data[range(20000,60000)].reshape([40000,28,28,1])
unlab_labels = labels[range(20000,60000)].reshape([40000,1])

In [183]:
test_data = test_data.reshape([10000,28,28,1])
test_labels = test_labels.reshape([10000,1])

## CNN v2.

A. Vulpe-Grigoraşi and O. Grigore, "Convolutional Neural Network Hyperparameters optimization for Facial Emotion Recognition," 2021 12th International Symposium on Advanced Topics in Electrical Engineering (ATEE), 2021, pp. 1-5, doi: 10.1109/ATEE52255.2021.9425073.

In [184]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [185]:
#######HYPERPARAMATERS###########
epochs = 100
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))
model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data,
          train_labels,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True
          )

Epoch 1/100
219/219 [==============================] - 4s 15ms/step - loss: 2.1904 - accuracy: 0.6861 - val_loss: 0.6671 - val_accuracy: 0.7623
Epoch 2/100
219/219 [==============================] - 3s 16ms/step - loss: 0.5788 - accuracy: 0.7938 - val_loss: 0.5860 - val_accuracy: 0.7912
Epoch 3/100
219/219 [==============================] - 3s 13ms/step - loss: 0.5037 - accuracy: 0.8167 - val_loss: 0.5621 - val_accuracy: 0.8002
Epoch 4/100
219/219 [==============================] - 3s 12ms/step - loss: 0.4599 - accuracy: 0.8329 - val_loss: 0.5134 - val_accuracy: 0.8253
Epoch 5/100
219/219 [==============================] - 3s 11ms/step - loss: 0.4145 - accuracy: 0.8510 - val_loss: 0.4916 - val_accuracy: 0.8230
Epoch 6/100
219/219 [==============================] - 3s 12ms/step - loss: 0.3904 - accuracy: 0.8533 - val_loss: 0.5009 - val_accuracy: 0.8268
Epoch 7/100
219/219 [==============================] - 2s 11ms/step - loss: 0.3636 - accuracy: 0.8610 - val_loss: 0.4925 - val_accuracy:

In [186]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

In [187]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

## 모델 결과

In [188]:
def model_eval():
  pred_mu = model.predict(test_data)
  for i in range(1, 30):
    pred_mu += model.predict(test_data)
  pred_mu = pred_mu/30
  predicted_test_labels = np.argmax(pred_mu, axis=1)
  return(accuracy_score(test_labels, predicted_test_labels))

In [189]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')
model_eval() #0.89xx

0.8981

# 라벨링

## 라벨링; 1

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

In [191]:
# Vars : 분산들
# Outs : 결과 값들

Vars = []
Outs = []

temp1 = []

for i in tqdm(range(30)):
  temp1.append(model.predict(unlab_data))

for j in tqdm(range(unlab_data.shape[0])):

  temp2 = np.array([0,0,0,0,0,0,0,0,0,0], 'float32')
  
  for i in range(30):
    temp2 += temp1[i][j]
  Outs.append(np.argmax(temp2))
  
  outputs=[]

  for i in range(30):
    outputs.append(temp1[i][j][np.argmax(temp2)])
  Vars.append(np.var(outputs))

100%|██████████| 40000/40000 [00:07<00:00, 5315.47it/s]


In [192]:
lowvars = []
ind = 0 

for i in Vars:
  if i < 1e-5:
    lowvars.append(ind)
  ind += 1

In [209]:
highvars = []
for i in range(unlab_data.shape[0]):
  if i not in lowvars:
    highvars.append(i)

In [193]:
# 같은 길이의 랜덤하게 고른 data들에 모델로 label 부여 후 정확도 측정 : 0.90x
randomindx = random.sample(range(40000), len(lowvars))
accuracy_score(np.array(Outs)[randomindx], unlab_labels[randomindx])

0.9064558629776021

In [194]:
# 저분산의 data들만 모아서 모델로 label 부여 후 정확도 측정 : 0.99x
accuracy_score(np.array(Outs)[lowvars], unlab_labels[lowvars])

0.9978232227759638

In [199]:
train_data = np.concatenate([train_data, unlab_data[lowvars]], axis=0)
train_labels = np.concatenate([train_labels, unlab_labels[lowvars]], axis=0)

In [218]:
shufindx = random.sample(range(train_data.shape[0]),train_data.shape[0])
train_data = train_data[shufindx]
train_labels = train_labels[shufindx]

In [212]:
unlab_data = unlab_data[highvars]
unlab_labels = unlab_labels[highvars]

## 모델링; 1

In [219]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [220]:
#######HYPERPARAMATERS###########
epochs = 100
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))
model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data,
          train_labels,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True
          )

Epoch 1/100
410/410 [==============================] - 6s 14ms/step - loss: 1.2060 - accuracy: 0.8210 - val_loss: 0.3612 - val_accuracy: 0.8748
Epoch 2/100
410/410 [==============================] - 5s 13ms/step - loss: 0.3321 - accuracy: 0.8844 - val_loss: 0.3173 - val_accuracy: 0.8889
Epoch 3/100
410/410 [==============================] - 5s 11ms/step - loss: 0.3030 - accuracy: 0.8967 - val_loss: 0.3121 - val_accuracy: 0.8933
Epoch 4/100
410/410 [==============================] - 5s 13ms/step - loss: 0.2707 - accuracy: 0.9037 - val_loss: 0.2941 - val_accuracy: 0.9007
Epoch 5/100
410/410 [==============================] - 5s 11ms/step - loss: 0.2480 - accuracy: 0.9125 - val_loss: 0.2856 - val_accuracy: 0.9071
Epoch 6/100
410/410 [==============================] - 6s 14ms/step - loss: 0.2385 - accuracy: 0.9149 - val_loss: 0.2745 - val_accuracy: 0.9065
Epoch 7/100
410/410 [==============================] - 5s 12ms/step - loss: 0.2179 - accuracy: 0.9200 - val_loss: 0.2930 - val_accuracy:

In [221]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')

In [223]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')
model_eval() #0.89xx

0.8939

## 라벨링; 2

In [226]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')

In [227]:
# Vars : 분산들
# Outs : 결과 값들

Vars = []
Outs = []

temp1 = []

for i in tqdm(range(30)):
  temp1.append(model.predict(unlab_data))

for j in tqdm(range(unlab_data.shape[0])):

  temp2 = np.array([0,0,0,0,0,0,0,0,0,0], 'float32')
  
  for i in range(30):
    temp2 += temp1[i][j]
  Outs.append(np.argmax(temp2))
  
  outputs=[]

  for i in range(30):
    outputs.append(temp1[i][j][np.argmax(temp2)])
  Vars.append(np.var(outputs))

100%|██████████| 22543/22543 [00:04<00:00, 5289.37it/s]


In [239]:
lowvars = []
ind = 0 

for i in Vars:
  if i < 1e-5:
    lowvars.append(ind)
  ind += 1

In [240]:
highvars = []
for i in range(unlab_data.shape[0]):
  if i not in lowvars:
    highvars.append(i)

In [241]:
# 같은 길이의 랜덤하게 고른 data들에 모델로 label 부여 후 정확도 측정 : 0.90x
randomindx = random.sample(range(unlab_data.shape[0]), len(lowvars))
accuracy_score(np.array(Outs)[randomindx], unlab_labels[randomindx])

0.8252979414951246

In [242]:
# 저분산의 data들만 모아서 모델로 label 부여 후 정확도 측정 : 0.99x
accuracy_score(np.array(Outs)[lowvars], unlab_labels[lowvars])

0.9886240520043337

In [243]:
len(highvars)

18851

In [244]:
train_data = np.concatenate([train_data, unlab_data[lowvars]], axis=0)
train_labels = np.concatenate([train_labels, unlab_labels[lowvars]], axis=0)

In [245]:
unlab_data = unlab_data[highvars]
unlab_labels = unlab_labels[highvars]

In [246]:
shufindx = random.sample(range(train_data.shape[0]),train_data.shape[0])
train_data = train_data[shufindx]
train_labels = train_labels[shufindx]

In [249]:
unlab_data.shape

(18851, 28, 28, 1)

## 모델링; 2

In [250]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [ ]:
#######HYPERPARAMATERS###########
epochs = 100
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))
model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data,
          train_labels,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True
          )

Epoch 1/100
451/451 [==============================] - 7s 14ms/step - loss: 0.9761 - accuracy: 0.8484 - val_loss: 0.3549 - val_accuracy: 0.8809
Epoch 2/100
451/451 [==============================] - 5s 11ms/step - loss: 0.3056 - accuracy: 0.8982 - val_loss: 0.3014 - val_accuracy: 0.9003
Epoch 3/100
451/451 [==============================] - 6s 12ms/step - loss: 0.2685 - accuracy: 0.9089 - val_loss: 0.2846 - val_accuracy: 0.9058
Epoch 4/100
451/451 [==============================] - 5s 11ms/step - loss: 0.2401 - accuracy: 0.9189 - val_loss: 0.2752 - val_accuracy: 0.9079
Epoch 5/100
451/451 [==============================] - 5s 11ms/step - loss: 0.2227 - accuracy: 0.9223 - val_loss: 0.2694 - val_accuracy: 0.9155
Epoch 6/100
451/451 [==============================] - 6s 12ms/step - loss: 0.2071 - accuracy: 0.9267 - val_loss: 0.2452 - val_accuracy: 0.9179
Epoch 7/100
451/451 [==============================] - 5s 11ms/step - loss: 0.1955 - accuracy: 0.9303 - val_loss: 0.2454 - val_accuracy:

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn3.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn3.h5')
model_eval() #0.89xx